In [1]:
'''
Data Cleaning and Preparation
Problem Statement: Analyzing Customer Churn in a Telecommunications Company
Dataset: "Telecom_Customer_Churn.csv"
Description: The dataset contains information about customers of a telecommunications
company and whether they have churned (i.e., discontinued their services). The dataset
includes various attributes of the customers, such as their demographics, usage patterns, and
account information. The goal is to perform data cleaning and preparation to gain insights
into the factors that contribute to customer churn.
Tasks to Perform:
1. Import the "Telecom_Customer_Churn.csv" dataset.
2. Explore the dataset to understand its structure and content.
3. Handle missing values in the dataset, deciding on an appropriate strategy.
4. Remove any duplicate records from the dataset.
5. Check for inconsistent data, such as inconsistent formatting or spelling variations,
and standardize it.
6. Convert columns to the correct data types as needed.
7. Identify and handle outliers in the data.
8. Perform feature engineering, creating new features that may be relevant to
predicting customer churn.
9. Normalize or scale the data if necessary.
10. Split the dataset into training and testing sets for further analysis.
11. Export the cleaned dataset for future analysis or modeling.
'''
None

import pandas as pd
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Import the "Telecom_Customer_Churn.csv" dataset.
data = pd.read_csv('Telco-Customer-Churn.csv')
data

# 2. Explore the dataset to understand its structure and content.
data.describe()

data.shape

data.info()

# 3. Handle missing values in the dataset, deciding on an appropriate strategy.
data.isnull().sum()

# 4. Remove any duplicate records from the dataset.
data.duplicated().sum()

# 5. Check for inconsistent data, such as inconsistent formatting or spelling variations,
# and standardize it.
data.columns

# obj = StandardScaler()
# data = data.target
# data = obj.fit_transform(data)
data

# 6. Convert columns to the correct data types as needed.
# gender_mapping = {'Male': 1, 'Female': 0}
# Partner_mapping = {'Yes': 1, 'No': 0}
# Dependents_mapping = {'Yes': 1, 'No': 0}
# PhoneService_mapping = {'Yes': 1, 'No': 0}
# MultipleLines_mapping = {'No phone service': 1, 'No': 0}
# PhoneService_mapping = {'Yes': 1, 'No': 0}
# MultipleLines_mapping = {'No phone service': 0, 'No': 0,'Yes':1}


# data['gender'] = data['gender'].map(gender_mapping)

data = data.drop(columns=['customerID'])

lb = LabelBinarizer()

# Convert 'Yes'/'No' binary columns using LabelBinarizer
binary_columns = ['Partner', 'Dependents', 'PhoneService', 'MultipleLines',
                  'OnlineSecurity', 'DeviceProtection', 'TechSupport', 
                  'StreamingTV', 'StreamingMovies', 'PaperlessBilling', 'Churn','OnlineBackup']

for col in binary_columns:
    data[col] = lb.fit_transform(data[col])

# Encode 'gender' as 1 for 'Female' and 0 for 'Male'
data['gender'] = lb.fit_transform(data['gender'])

# Convert 'InternetService' to numerical values
data['InternetService'] = data['InternetService'].map({'DSL': 1, 'Fiber optic': 2, 'No': 0})

# Convert 'Contract' and 'PaymentMethod' using one-hot encoding
data = pd.get_dummies(data, columns=['Contract', 'PaymentMethod'], drop_first=True)

# Convert 'TotalCharges' to numeric, handle errors due to empty strings
data['TotalCharges'] = pd.to_numeric(data['TotalCharges'], errors='coerce')

binary_columns_new = ['Contract_One year','Contract_Two year','PaymentMethod_Credit card (automatic)','PaymentMethod_Electronic check','PaymentMethod_Mailed check']

for col in binary_columns_new:
    data[col] = lb.fit_transform(data[col])
data

data.dtypes

# 7. Identify and handle outliers in the data.
numeric_columns = data.columns

for col in numeric_columns:
    Q1 = data[col].quantile(0.25)
    Q3 = data[col].quantile(0.75)
    IQR = Q3 - Q1
    
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Filter out rows with outliers
    outliers = data[(data[col] < lower_bound) | (data[col] > upper_bound)]
    print(f'Outliers detected in {col}: {len(outliers)}')

    # handling outliers
for col in numeric_columns:
    Q1 = data[col].quantile(0.25)
    Q3 = data[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Remove outliers
    data = data[(data[col] >= lower_bound) & (data[col] <= upper_bound)]
    outliers = data[(data[col] < lower_bound) | (data[col] > upper_bound)]
    print(f'Outliers detected in {col}: {len(outliers)}')

    # 8. Perform feature engineering, creating new features that may be relevant to
# predicting customer churn.
def tenure_group(tenure):
    if tenure <= 12:
        return '0-1 year'
    elif tenure <= 24:
        return '1-2 years'
    elif tenure <= 48:
        return '2-4 years'
    elif tenure <= 60:
        return '4-5 years'
    else:
        return '5+ years'

data['TenureGroup'] = data['tenure'].apply(tenure_group)
data['ChargesRatio'] = data['TotalCharges'] / (data['MonthlyCharges'] + 1)  # Adding 1 to avoid division by zero
data

# 10. Split the dataset into training and testing sets for further analysis.
# 11. Export the cleaned dataset for future analysis or modeling.

from sklearn.model_selection import train_test_split

# Define your target variable and features
X = data.drop(columns=['Churn'])  # Features (drop 'Churn' as it’s the target)
y = data['Churn']  # Target variable

# Split the data (80% for training, 20% for testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Training set size: {X_train.shape}")
print(f"Testing set size: {X_test.shape}")

data.to_csv('cleaned_churn_data.csv', index=False)

# Optionally, export training and testing sets separately
X_train.to_csv('X_train.csv', index=False)
X_test.to_csv('X_test.csv', index=False)
y_train.to_csv('y_train.csv', index=False)
y_test.to_csv('y_test.csv', index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


In [ ]:
# 9. Normalize or scale the data if necessary(Not required).

In [ ]:
# Done